In [1]:

import os
import os.path as osp
import shutil
import json
import pickle
import collections as C
import itertools as I
import random
from typing import Optional

from pprint import pprint
from math_verify import parse
from tqdm import tqdm

from common.utils import extract_code, replace_sorry

In [2]:
def print_xtuner_sample(data: list):
    if 'system' in data['conversation'][0].keys():
        print('<SYSTEM>')
        print(data['conversation'][0]['system'])
        print('</SYSTEM>')
    print('<INPUT>')
    print(data['conversation'][0]['input'])
    print('</INPUT>\n<OUTPUT>')
    print(data['conversation'][0]['output'])
    print('</OUTPUT>')


In [3]:
data_root = '/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/data/informalization/informal_6'
file_name = 'autoformalization_pg_kimina7b-ScaleQuest-Math_kimina7b-valid_samples_output.jsonl'

In [5]:
def parse_informal_output(d: dict):
    try:
        d |= json.loads(extract_code(d['informal_output']))
    except:
        pass

In [3]:
with open(osp.join(data_root, file_name), 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
len(data), data[0]

(2035,
 {'header': '',
  'formal_statement': 'example : 16 / 4 = 4 := sorry',
  'formal_proof': '\n  have h_main : 16 / 4 = 4 := by\n    norm_num\n    <;> rfl\n    <;> norm_num\n    <;> rfl\n  -- The `norm_num` tactic is used to simplify the expression and verify the result.\n  -- The `rfl` tactic is used to confirm that the simplified form matches the expected result.\n  exact h_main',
  'condition': 0,
  'index': 0,
  'src': 'autoformalization_pg_kimina7b/ScaleQuest-Math_kimina7b',
  'informal_output': '```json\n{\n  "informal_problem": "Show that 16 divided by 4 equals 4.",\n  "problem_type": "proof problem",\n  "informal_answer": "None"\n}\n```'})

In [6]:
for d in data:
    parse_informal_output(d)

In [7]:
C.Counter(d.get('problem_type') for d in data)

Counter({'proof problem': 1716, 'solution problem': 317, None: 2})

In [115]:
d = random.choice([d for d in data if d.get('problem_type') == 'proof problem'])
pprint(d['formal_statement'])
pprint(d['formal_proof'])
pprint(d.get('problem_type'))
pprint(d.get('informal_problem'))

'example (x : ℝ) (hx : x = 0.95 * 1.06) :\n    (x - 1) * 100 = 0.7 := sorry'
('\n'
 '  have h₁ : x = 1.007 := by\n'
 '    rw [hx]\n'
 '    norm_num [mul_comm]\n'
 '    <;> ring_nf\n'
 '    <;> norm_num\n'
 '    <;> linarith\n'
 '  \n'
 '  have h₂ : (x - 1) * 100 = 0.7 := by\n'
 '    rw [h₁]\n'
 '    norm_num [mul_comm]\n'
 '    <;> ring_nf\n'
 '    <;> norm_num\n'
 '    <;> linarith\n'
 '  \n'
 '  exact h₂')
'proof problem'
('Given a real number x such that x = 0.95 * 1.06, show that (x - 1) * 100 = '
 '0.7.')


In [80]:
demonstrations = {
    r'''example
(x : ℝ)
(hx : 0 < x)
(y : ℝ)
(hy : 0 < y)
(z : ℝ)
(hz : 0 < z)
(k : ℝ)
(h : x * y * z * k = 1)
(hk : 0 < k)
: x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) ≥ 4
:= by
have h1 : 0 < x * y * z + y * z * k + z * x * k + x * y * k := by positivity

have h2 : x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) = (x ^ 2 + y ^ 2 + z ^ 2 + k ^ 2) / (x * y * z * k) :=
  by
  field_simp
  ring

have h3: 
    x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) ≥ 4  := by {
  rw [h2]
  rw [h]
  nlinarith [sq_nonneg (x - y), sq_nonneg (x - z), sq_nonneg (x - k), sq_nonneg (y - z), sq_nonneg (y - k), sq_nonneg (z - k),
    mul_pos hx hy, mul_pos hx hz, mul_pos hx hk, mul_pos hy hz, mul_pos hy hk, mul_pos hz hk]
}

exact h3''' : 
    {
        'problem_type': 'Proof Question',
        'informal_statement': r'''Let \(x, y, z, k\) be positive real numbers such that \(x \cdot y \cdot z \cdot k = 1\). Prove that:

\[
\frac{x}{y \cdot z \cdot k} + \frac{y}{z \cdot x \cdot k} + \frac{z}{x \cdot y \cdot k} + \frac{k}{x \cdot y \cdot z} \geq 4.
\]''',
        'informal_proof': r'''**Step 1.** Since \(x, y, z, k > 0\), the denominator \(x \cdot y \cdot z + y \cdot z \cdot k + z \cdot x \cdot k + x \cdot y \cdot k\) is positive (by the positivity of the terms).

**Step 2.** We simplify the left-hand side by combining the fractions:

\[
\begin{aligned}
&\frac{x}{y \cdot z \cdot k} + \frac{y}{z \cdot x \cdot k} + \frac{z}{x \cdot y \cdot k} + \frac{k}{x \cdot y \cdot z} \\
&= \frac{x^2 + y^2 + z^2 + k^2}{x \cdot y \cdot z \cdot k}.
\end{aligned}
\]

This is verified by using a common denominator \(x \cdot y \cdot z \cdot k\) and simplifying the numerator.

**Step 3.** Since \(x \cdot y \cdot z \cdot k = 1\), the expression becomes:

\[
x^2 + y^2 + z^2 + k^2.
\]

**Step 4.** To prove \(x^2 + y^2 + z^2 + k^2 \geq 4\), we use the non-negativity of squares:

\[
(x - y)^2 \geq 0, \quad (x - z)^2 \geq 0, \quad (x - k)^2 \geq 0,
\]
\[
(y - z)^2 \geq 0, \quad (y - k)^2 \geq 0, \quad (z - k)^2 \geq 0.
\]

Expanding and summing these inequalities (along with using the fact that \(x, y, z, k > 0\)) leads to:

\[
3(x^2 + y^2 + z^2 + k^2) \geq 2(xy + xz + xk + yz + yk + zk).
\]

By the AM–GM inequality (or further algebraic manipulation), we conclude:

\[
x^2 + y^2 + z^2 + k^2 \geq 4.
\]

Thus, the original inequality holds.'''
    },
    
    r'''example
(z : ℂ)
(hz : z = 2 + 3 * .I)
(w : ℂ)
(hw : w = 2 - 3 * .I)
: z * w = 13
:= by
have h: 
    z * w = 13  := by {
  rw [hz, hw]
  simp [Complex.ext_iff, Complex.add_re, Complex.add_im, Complex.mul_re, Complex.mul_im, Complex.I_re, Complex.I_im]
  all_goals norm_num
}

exact h''' :
    {
        'problem_type': 'Problem-Solving Question',
        'informal_problem': r'''Let \( z \) and \( w \) be complex numbers such that \( z = 2 + 3i \) and \( w = 2 - 3i \). Compute the product \( z \times w \).''',
        'informal_answer': r'''13''',
        'informal_solution': r'''**Step 1: Write down the given expressions.**  
Given:  
\[
z = 2 + 3i, \quad w = 2 - 3i.
\]

**Step 2: Compute the product \( z \times w \).**  
\[
z \times w = (2 + 3i)(2 - 3i).
\]

**Step 3: Expand the product using the distributive property (FOIL method).**  
\[
(2 + 3i)(2 - 3i) = 2 \cdot 2 + 2 \cdot (-3i) + 3i \cdot 2 + 3i \cdot (-3i).
\]  
Simplify each term:  
\[
= 4 - 6i + 6i - 9i^2.
\]

**Step 4: Combine like terms and simplify using \( i^2 = -1 \).**  
The imaginary terms \(-6i\) and \(+6i\) cancel each other:  
\[
= 4 + 0i - 9i^2 = 4 - 9i^2.
\]  
Substitute \( i^2 = -1 \):  
\[
= 4 - 9(-1) = 4 + 9 = 13.
\]

**Step 5: Conclude the result.**  
Thus,  
\[
z \times w = 13.
\]

**Final Answer:**  
\[
\boxed{13}
\]'''
    },
    r'''example
(a : ℕ → ℤ)
(ha1 : a 1 = 2)
(han : ∀ n ≥ 1, a (n + 1) = a n + 2 * n)
: a 100 = 9902
:= by
have h: 
    a 100 = 9902  := by {
  norm_num [ha1, han] at *
  <;> try { all_goals omega }
}

exact h''' :
    {
        'problem_type': 'Problem-Solving Question',
        'informal_problem': r'''Define a sequence \( a_1, a_2, a_3, \ldots \) of integers by:
- \( a_1 = 2 \), and
- For every integer \( n \geq 1 \), \( a_{n+1} = a_n + 2n \).

Calculate the value of \( a_{100} \).''',
        'informal_answer': r'''9902''',
        'informal_solution': r'''1.  **Understand the Recurrence Relation:**
    The sequence is defined by:
    \[
    a_1 = 2
    \]
    \[
    a_{n+1} = a_n + 2n \quad \text{for all } n \geq 1
    \]
    This means each term is obtained by adding \( 2n \) (which is twice the current index \( n \)) to the previous term.

2.  **Write Out the First Few Terms:**
    Let's compute the first few terms to identify a pattern.
    \[
    \begin{align*}
    a_1 &= 2 \\
    a_2 &= a_1 + 2 \cdot 1 = 2 + 2 = 4 \\
    a_3 &= a_2 + 2 \cdot 2 = 4 + 4 = 8 \\
    a_4 &= a_3 + 2 \cdot 3 = 8 + 6 = 14 \\
    a_5 &= a_4 + 2 \cdot 4 = 14 + 8 = 22 \\
    \end{align*}
    \]
    The sequence begins: \( 2, 4, 8, 14, 22, \ldots \)

3.  **Look for a Pattern or Closed Form:**
    Observe the differences between consecutive terms:
    \[
    a_2 - a_1 = 2 = 2 \cdot 1 \\
    a_3 - a_2 = 4 = 2 \cdot 2 \\
    a_4 - a_3 = 6 = 2 \cdot 3 \\
    a_5 - a_4 = 8 = 2 \cdot 4 \\
    \]
    This confirms the recurrence \( a_{n+1} - a_n = 2n \).

    To find a closed form for \( a_n \), we can express \( a_n \) as the sum of the first term and all the subsequent differences:
    \[
    \begin{align*}
    a_n &= a_1 + \sum_{k=1}^{n-1} (a_{k+1} - a_k) \\
        &= 2 + \sum_{k=1}^{n-1} 2k \\
        &= 2 + 2 \sum_{k=1}^{n-1} k
    \end{align*}
    \]

4.  **Apply the Formula for the Sum of the First \( m \) Integers:**
    The well-known formula is \( \sum_{k=1}^{m} k = \frac{m(m+1)}{2} \). Here, the upper limit is \( m = n-1 \), so:
    \[
    \sum_{k=1}^{n-1} k = \frac{(n-1)n}{2}
    \]
    Substituting this back into our expression for \( a_n \):
    \[
    \begin{align*}
    a_n &= 2 + 2 \cdot \left( \frac{(n-1)n}{2} \right) \\
        &= 2 + (n-1)n \\
        &= n^2 - n + 2
    \end{align*}
    \]
    We have derived the closed-form formula:
    \[
    a_n = n^2 - n + 2
    \]
    Let's verify this with the initial terms:
    \[
    a_1 = 1^2 - 1 + 2 = 2 \quad \checkmark \\
    a_2 = 4 - 2 + 2 = 4 \quad \checkmark \\
    a_3 = 9 - 3 + 2 = 8 \quad \checkmark \\
    a_4 = 16 - 4 + 2 = 14 \quad \checkmark \\
    \]
    The formula holds.

5.  **Calculate \( a_{100} \):**
    Substitute \( n = 100 \) into the closed-form formula:
    \[
    \begin{align*}
    a_{100} &= 100^2 - 100 + 2 \\
            &= 10000 - 100 + 2 \\
            &= 9900 + 2 \\
            &= 9902
    \end{align*}
    \]

**Conclusion:** The 100th term of the sequence is \( 9902 \).

\[
\boxed{9902}
\]'''
    },
    r'''example
(a : ℝ)
(ha : 0 < a)
(b : ℝ)
(hb : 0 < b)
(f : ℝ → ℝ)
(hf : ∀ x, f x = a * x ^ 4 - b * x ^ 2 + x + 7)
(h : f (-5) = 3)
: f 5 = 13
:= by
have eq1 : a * (-5) ^ 4 - b * (-5) ^ 2 + (-5) + 7 = 3 := by
  rw [hf] at h
  linarith

have eq2 : 625 * a - 25 * b = 1 := by nlinarith

have eq3 : 25 * b = 625 * a - 1 := by linarith [eq2]

have eq4 : b = 25 * a - 1 / 25 := by linarith

have h:  f 5 = 13  := by {
  rw [hf 5]
  rw [show b = 25 * a - 1 / 25 by linarith [eq4] ]
  nlinarith [ sq_nonneg (a), sq_nonneg (b), 
          sq_nonneg (a - 1), sq_nonneg (b - 1), 
          sq_nonneg (a * b), sq_nonneg (a * b - 1)]
}

exact h''' : 
    {
        'problem_type': 'Problem-Solving Question',
        'informal_problem': r'''Let \( a \) and \( b \) be positive real numbers. Define a function \( f : \mathbb{R} \to \mathbb{R} \) by  
\[
f(x) = a \cdot x^4 - b \cdot x^2 + x + 7.
\]  
Given that \( f(-5) = 3 \), what is the value of \( f(5) \)?''',
        'informal_answer': r'''13''',
        'informal_solution': r'''**Step 1:** Substitute \( x = -5 \) into \( f(x) \).  
Since \( f(-5) = 3 \), we have:  
\[
a (-5)^4 - b (-5)^2 + (-5) + 7 = 3.
\]  
Simplifying the powers:  
\[
625a - 25b - 5 + 7 = 3.
\]  
Combine constants:  
\[
625a - 25b + 2 = 3.
\]  
Subtract 2 from both sides:  
\[
625a - 25b = 1. \tag{1}
\]

**Step 2:** Solve equation (1) for \( b \).  
Rewriting:  
\[
25b = 625a - 1.
\]  
Divide by 25:  
\[
b = 25a - \frac{1}{25}. \tag{2}
\]

**Step 3:** Compute \( f(5) \).  
\[
f(5) = a (5)^4 - b (5)^2 + 5 + 7 = 625a - 25b + 12.
\]  
Substitute \( b \) from equation (2):  
\[
f(5) = 625a - 25\left(25a - \frac{1}{25}\right) + 12.
\]  
Simplify:  
\[
f(5) = 625a - 625a + 1 + 12 = 13.
\]

Therefore, \( f(5) = 13 \), as required.

---

### Final Answer:
\[
\boxed{13}
\]'''
    },
    r'''example
(g : ℕ → ℕ)
(hg : ∀ n, 0 < g n)
(h1 : ∀ m n, g (m + n) = g m * g n)
(h2 : ∀ n, g n ∣ n)
: ∀ (n : ℕ), g n = 1
:= by
have h3 := h2 0

have h7 : g 0 = 1 := by
  have h10 : g 0 ∣ 0 := by
    use 0
    all_goals
      simp
  have h11 : g 0 = 1 := by
    have h6 : 0 < g 0 := by
      apply hg 0
    have h12 : g 0 ∣ 0 := h10
    have h13 : g 0 < 0 ∨ g 0 = 0 ∨ 0 < g 0 := by
      omega
    rcases h13 with (h | h | h)
    · 
      exfalso
      have h14 : 0 < g 0 := by
        apply hg 0
      omega
    · 
      exfalso
      have h14 : 0 < g 0 := by
        apply hg 0
      omega
    · 
      have h14 : 0 < g 0 := h
      have h15 : g 0 = 1 := by
        by_contra h17
        push_neg at h17
        have h18 : g 0 > 1 := by
          omega
        have h19 : ∃ x, g 0 = x + 1 := by
          refine ⟨g 0 - 1, by omega⟩
        rcases h19 with ⟨x, h18⟩
        have h20 : g 0 ∣ 0 := h10
        have h21 : ∃ k, g 0 * k = 0 := by
          rcases h20 with ⟨k, hk⟩
          use k
          linarith
        rcases h21 with ⟨k, hk⟩
        have h22 : g 0 * k = 0 := hk
        have h23 : g 0 > 0 := by
          linarith
        have h24 : k = 0 := by
          nlinarith
        have h25 : g (0 + 0) = g 0 * g 0 := h1 0 0
        have h26 : g (0 + 0) = g 0 := by
          simp
        rw [h26] at h25
        rw [h24] at h22
        nlinarith
      exact h15
  exact h11

have h4 : g 1 ∣ 1 := h2 1

have h8 : g 1 = 1 := by
  have h9 : g 1 ∣ 1 := h4
  have h10 : 0 < g 1 := hg 1
  have h11 : g 1 ≤ 1 := by exact Nat.le_of_dvd (by norm_num) h9
  have h12 : g 1 ≥ 1 := by linarith [h10]
  linarith

have h9: 
    ∀ n, g n = 1  := by {
  intro n 
  have h30 : ∀ n, g n = 1 := by 
    intro n 
    induction' n using Nat.strongRecOn with n ih
    cases n with
    | zero =>
      exact h7 
    | succ n =>
      cases n with
      | zero =>
        exact h8 
      | succ n =>
        have h9 : g (n + 1 + 1) = g (n + 1) * g 1 := by 
          specialize h1 (n + 1) 1 
          exact h1 
        rw [h9]
        rw [show g (n + 1) = 1 by apply ih (n + 1) (by omega)]
        rw [show g 1 = 1 by exact h8]
        all_goals
          norm_num
  apply h30 n
}

exact h9''' : 
    {
        'problem_type': 'Problem-Solving Question',
        'informal_problem': r'''Let \( g \) be a function from natural numbers to natural numbers such that:
1. For every \( n \), \( g(n) > 0 \).
2. For every \( m \) and \( n \), \( g(m + n) = g(m) \cdot g(n) \).
3. For every \( n \), \( g(n) \) divides \( n \).

Determine the value of \( g(n) \) for every natural number \( n \).''',
        'informal_answer': r'''1''',
        'informal_solution': r'''#### Step 1: Show \( g(0) = 1 \)
From condition 3, \( g(0) \mid 0 \), so there exists \( k \) such that \( g(0) \cdot k = 0 \). Since \( g(0) > 0 \) by condition 1, we must have \( k = 0 \), so \( g(0) \cdot 0 = 0 \).  
Now, using condition 2 with \( m = 0 \) and \( n = 0 \):
\[
g(0 + 0) = g(0) \cdot g(0) \implies g(0) = g(0)^2.
\]
Since \( g(0) > 0 \), we conclude \( g(0) = 1 \).

#### Step 2: Show \( g(1) = 1 \)
From condition 3, \( g(1) \mid 1 \), so \( g(1) = 1 \) (since \( g(1) > 0 \)).

#### Step 3: Prove \( g(n) = 1 \) for all \( n \) by strong induction
- **Base cases**:  
  \( g(0) = 1 \) and \( g(1) = 1 \) as shown.
- **Inductive step**:  
  Assume \( g(k) = 1 \) for all \( k < n \). We show \( g(n) = 1 \).  
  Write \( n = (n-1) + 1 \). Then by condition 2:
  \[
  g(n) = g(n-1) \cdot g(1) = g(n-1) \cdot 1 = g(n-1).
  \]
  Since \( n-1 < n \), by the inductive hypothesis \( g(n-1) = 1 \), so \( g(n) = 1 \).

Thus, by strong induction, \( g(n) = 1 \) for all \( n \).

\[
\boxed{1}
\]
'''
    }
}


In [124]:
# PTYPE_DICT = {
#     'Problem-Solving Question': 'Problem-Solving Question',
#     'Proof Question': 'Proof Question'
# } 


def format_informalization_response(
    problem_type: str,
    informal_problem: str,
    informal_solution: str,
    informal_answer: Optional[str]
) -> str:
    assert all(split not in field
        for split in [f'## Problem-Solving Question', '## Proof Question', '## Answer', '## Solution', '## Proof']
        for field in [problem_type, informal_problem, informal_solution, (informal_answer or '')]
    )
    response = f'## {problem_type.strip()}\n{informal_problem.strip()}\n\n'
    if problem_type == 'Problem-Solving Question':
        assert informal_answer is not None
        response += f'## Answer\n{informal_answer.strip()}\n\n'
        response += f'## Solution\n{informal_solution.strip()}\n\n'
    elif problem_type == 'Proof Question':
        response += f'## Proof\n{informal_solution.strip()}\n\n'
    else:
        raise ValueError(f'Invalid problem_type: "{problem_type}"')
    return response


DEMONSTRATION_TEXT = ''
for code, d in demonstrations.items():
    DEMONSTRATION_TEXT += f'''## Formal Code
```lean4
{code}
```

{format_informalization_response(problem_type=d['problem_type'], informal_problem=(d.get('informal_problem') or d.get('informal_statement')), informal_solution=d.get('informal_solution') or d.get('informal_proof'), informal_answer=d.get('informal_answer')).strip()}

---

'''

In [125]:
print(DEMONSTRATION_TEXT)

## Formal Code
```lean4
example
(x : ℝ)
(hx : 0 < x)
(y : ℝ)
(hy : 0 < y)
(z : ℝ)
(hz : 0 < z)
(k : ℝ)
(h : x * y * z * k = 1)
(hk : 0 < k)
: x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) ≥ 4
:= by
have h1 : 0 < x * y * z + y * z * k + z * x * k + x * y * k := by positivity

have h2 : x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) = (x ^ 2 + y ^ 2 + z ^ 2 + k ^ 2) / (x * y * z * k) :=
  by
  field_simp
  ring

have h3: 
    x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) ≥ 4  := by {
  rw [h2]
  rw [h]
  nlinarith [sq_nonneg (x - y), sq_nonneg (x - z), sq_nonneg (x - k), sq_nonneg (y - z), sq_nonneg (y - k), sq_nonneg (z - k),
    mul_pos hx hy, mul_pos hx hz, mul_pos hx hk, mul_pos hy hz, mul_pos hy hk, mul_pos hz hk]
}

exact h3
```

## Proof Question
Let \(x, y, z, k\) be positive real numbers such that \(x \cdot y \cdot z \cdot k = 1\). Prove that:

\[
\frac{x}{y \cdot z \cdot k} + \frac{y}{z \cdot x \cdot k} + \f

In [174]:
def format_informalization_prompt(header: Optional[str], formal_statement: str, formal_proof: str) -> str:
    header = (header or '').strip()
    formal_statement = replace_sorry(formal_statement)
    assert formal_statement.endswith(':= sorry')
    if formal_proof.startswith('\n'):
        formal_proof = formal_proof[1:]
    formal_code = header + '\n' + formal_statement[:-len('sorry')] + 'by\n' + formal_proof
    return f'''Given a Lean 4 formal statement and its proof, please translate them into natural language.
1. Determine its question type: "Problem-Solving Question" or "Proof Question". Prefer "Problem-Solving Question" if possible.
2. If it is a "Problem-Solving Question", please translate the formal code into a natural language question, its answer, and its solution. The answer should not appear in the question text. The solution should wrap the final answer with "\\boxed{{}}".
3. If it is a "Proof Question", please translate it into a natural language proposition and its proof.
4. Please maintain the semantic equivalence between the natural language question+answer/proposition and the formal statement, and between the solution/proof and the formal proof.
5. Please reply in markdown format with level-2 headers such as "## Problem-Solving Question", "## Answer" and "## Solution".

The following are some examples:

{DEMONSTRATION_TEXT}Now, please translate the following formal statement and its proof.

## Formal Code
```lean4
{formal_code.strip()}
```
'''

In [127]:
print(format_informalization_prompt(
    header=d['header'],
    formal_statement=d['formal_statement'],
    formal_proof=d['formal_proof']
))

KeyError: 'header'

In [128]:
for d in data:
    prompt = format_informalization_prompt(
        header=d['header'],
        formal_statement=d['formal_statement'],
        formal_proof=d['formal_proof']
    )

In [129]:
print(prompt)

Given a Lean 4 formal statement and its proof, please translate them into natural language.
1. Determine its question type: "Problem-Solving Question" or "Proof Question". Prefer "Problem-Solving Question" if possible.
2. If it is a "Problem-Solving Question", please translate the formal code into a natural language question, its answer, and its solution. The answer should not appear in the question text. The solution should wrap the final answer with "\boxed{}".
3. If it is a "Proof Question", please translate it into a natural language proposition and its proof.
4. Please maintain the semantic equivalence between the natural language question+answer/proposition and the formal statement, and between the solution/proof and the formal proof.

The following are some examples:

## Formal Code
```lean4
example
(x : ℝ)
(hx : 0 < x)
(y : ℝ)
(hy : 0 < y)
(z : ℝ)
(hz : 0 < z)
(k : ℝ)
(h : x * y * z * k = 1)
(hk : 0 < k)
: x / (y * z * k) + y / (z * x * k) + z / (x * y * k) + k / (x * y * z) ≥ 

In [ ]:
response = r'''## Problem-Solving Question
Compute the sum \( \left(1 + \frac{1}{2}\right) + \left(2 + \frac{1}{3}\right) \), where the numbers are rational.

## Answer
\(\frac{23}{6}\)

## Solution
**Step 1:** Write the expression clearly:
\[
\left(1 + \frac{1}{2}\right) + \left(2 + \frac{1}{3}\right)
\]

**Step 2:** Simplify each mixed number:
\[
1 + \frac{1}{2} = \frac{2}{2} + \frac{1}{2} = \frac{3}{2}
\]
\[
2 + \frac{1}{3} = \frac{6}{3} + \frac{1}{3} = \frac{7}{3}
\]

**Step 3:** Add the two fractions:
\[
\frac{3}{2} + \frac{7}{3}
\]

**Step 4:** Find a common denominator (which is 6):
\[
\frac{3}{2} = \frac{3 \times 3}{2 \times 3} = \frac{9}{6}
\]
\[
\frac{7}{3} = \frac{7 \times 2}{3 \times 2} = \frac{14}{6}
\]

**Step 5:** Add the numerators:
\[
\frac{9}{6} + \frac{14}{6} = \frac{23}{6}
\]

**Final Answer:**
\[
\boxed{\frac{23}{6}}
\]'''

In [10]:
# Parse response

def parse_response(response: str) -> dict:
    if '## Problem-Solving Question' in response:
        assert '\n## Answer' in response
        assert '\n## Solution' in response
        assert '## Proof Question' not in response
        assert '\n## Proof' not in response
        intro, body = response.split('## Problem-Solving Question')
        informal_problem, informal_answer_solution = body.split('\n## Answer')
        informal_answer, informal_solution = informal_answer_solution.split('\n## Solution')
        return dict(
            problem_type='Problem-Solving Question',
            informal_problem=informal_problem.strip(),
            informal_answer=informal_answer.strip(),
            informal_solution=informal_solution.strip()
        )
    else:
        assert '## Proof Question' in response
        assert '\n## Proof' in response
        assert '## Problem-Solving Question' not in response
        assert '\n## Answer' not in response
        assert '\n## Solution' not in response
        intro, body = response.split('## Proof Question')
        informal_statement, informal_proof = body.split('\n## Proof')
        return dict(
            problem_type='Proof Question',
            informal_problem=informal_statement.strip(),
            informal_answer=None,
            informal_solution=informal_proof.strip()
        )

In [148]:
# Sanity Check
for code, d in demonstrations.items():
    response = format_informalization_response(
        problem_type=d['problem_type'],
        informal_problem=(d.get('informal_problem') or d.get('informal_statement')),
        informal_solution=(d.get('informal_solution') or d.get('informal_proof')),
        informal_answer=d.get('informal_answer')
    )
    result = parse_response(response)
    assert result['problem_type'] == d['problem_type']
    assert result['informal_problem'].strip() == (d.get('informal_problem') or d.get('informal_statement')).strip()
    assert result['informal_solution'].strip() == (d.get('informal_solution') or d.get('informal_proof')).strip()
    assert result['informal_answer'] == d.get('informal_answer')

## Fix Failed Results

In [13]:
data_root = '/home/ma-user/workspace/formal_problem_generation/output_tmp/output/informalizations/processed'
for p in os.listdir(data_root):
    if osp.isfile(osp.join(data_root, p)):
        with open(osp.join(data_root, p), 'r') as f:
            data = [json.loads(l) for l in f.readlines()]
        if any('informalization' not in d.keys() for d in data):
            shutil.move(osp.join(data_root, p), osp.join(data_root, 'raw', p))
            print(p, len([d for d in data if 'informalization' not in d.keys()]))

In [14]:
# Start from here

In [15]:
if len(set(os.listdir(osp.join(data_root, 'raw'))) - set(os.listdir(data_root))) == 0:
    raise
for p in set(os.listdir(osp.join(data_root, 'raw'))) - set(os.listdir(data_root)):
    with open(osp.join(data_root, 'raw', p), 'r') as f:
        data = [json.loads(l) for l in f.readlines()]
    fail_list = [d for d in data if 'informalization' not in d.keys()]
    print(p, len(data), len(fail_list))
    break

RuntimeError: No active exception to reraise

In [6]:
fail_list = [d for d in data if 'informalization' not in d.keys()]
print(len(fail_list))

1


In [7]:
for d in fail_list:
    print(d['formal_statement'])
    print(d['responses'][0])
    print('\n--SEP--\n')

def f (x : ℝ) : ℝ :=
  2 * x + 3

example : f 4 = 11 := by
  unfold f
  norm_num


theorem f_increasing : ∀ a b : ℝ, a < b → f a < f b  := by
  sorry
## Problem-Solving Question
Given the function \( f(x) = 2x + 3 \), evaluate \( f(4) \).

## Answer
\boxed{11}

## Solution
The function is defined as \( f(x) = 2x + 3 \). To evaluate \( f(4) \), substitute \( x = 4 \) into the function:
\[
f(4) = 2 \cdot 4 + 3 = 8 + 3 = 11
\]
Thus, \( f(4) = \boxed{11} \).

---

## Proof Question
The function \( f(x) = 2x + 3 \) is strictly increasing on the real numbers. That is, for all real numbers \( a \) and \( b \), if \( a < b \), then \( f(a) < f(b) \).

## Proof
Let \( a \) and \( b \) be real numbers such that \( a < b \). We compute:
\[
f(a) = 2a + 3, \quad f(b) = 2b + 3.
\]
Since \( a < b \), multiplying both sides by 2 (a positive number) gives \( 2a < 2b \). Adding 3 to both sides preserves the inequality, yielding:
\[
2a + 3 < 2b + 3,
\]
which is exactly \( f(a) < f(b) \). Therefore, \( f 

In [8]:
modified_str = \
r'''def f (x : ℝ) : ℝ :=
  2 * x + 3

example : f 4 = 11 := by
  unfold f
  norm_num


theorem f_increasing : ∀ a b : ℝ, a < b → f a < f b  := by
  sorry
## Proof Question
Prove that \( f(x) = 2x + 3 \) is strictly increasing on the real numbers. That is, for all real numbers \( a \) and \( b \), if \( a < b \), then \( f(a) < f(b) \).

## Proof
Let \( a \) and \( b \) be real numbers such that \( a < b \). We compute:
\[
f(a) = 2a + 3, \quad f(b) = 2b + 3.
\]
Since \( a < b \), multiplying both sides by 2 (a positive number) gives \( 2a < 2b \). Adding 3 to both sides preserves the inequality, yielding:
\[
2a + 3 < 2b + 3,
\]
which is exactly \( f(a) < f(b) \). Therefore, \( f \) is strictly increasing. ∎
'''

In [11]:
chunks = modified_str.strip().split('\n--SEP--\n')
assert len(chunks) == len(fail_list)
for (c, d) in zip(chunks, fail_list):
    c = c.strip()
    assert c.startswith(d['formal_statement'])
    result = parse_response(c[len(d['formal_statement']):])
    d['informalization'] = result    

In [12]:
with open(osp.join(data_root, p), 'w') as f:
    for d in data:
        f.write(json.dumps(d, ensure_ascii=False) + '\n')

In [18]:
raise

RuntimeError: No active exception to reraise

## Check Problem Type

In [122]:
answer_succeeded_all = []
answer_failed_all = []

data_root = '/home/ma-user/workspace/formal_problem_generation/output_tmp/output/informalizations/processed'
for p in os.listdir(data_root):
    if osp.isfile(osp.join(data_root, p)):
        print(p)
        with open(osp.join(data_root, p), 'r') as f:
            data = [json.loads(l) for l in f.readlines()]

        potentially_wrong_list = [d for d in data if d['informalization']['problem_type'] == 'Problem-Solving Question' and  \
            any(t in d['informalization']['informal_problem'].lower() for t in ['prove', 'show'])]
        print(f'len(potentially_wrong_list)={len(potentially_wrong_list)}')
        for d in potentially_wrong_list:
            d['informalization']['problem_type'] = 'Proof Question'
        
        for d in data:
            if d['informalization']['problem_type'] == 'Problem-Solving Question':
                informal_answer = d['informalization']['informal_answer'].strip()
                if informal_answer.startswith(r'\boxed{'):
                    informal_answer = informal_answer[len(r'\boxed{'):-1]
                answer_parsed = parse(informal_answer)
                
                if len(answer_parsed) == 2 and any(t in str(type(answer_parsed[0]).__module__) for t in ['sympy.core.numbers', 'latex2sympy2_extended.sets', 'sympy.sets', 'sympy.core.mul', 'sympy.core.add', 'sympy.core.power', 'sympy.matrices', 'sympy.functions']):
                    assert isinstance(answer_parsed[-1], str)
                    d['informalization']['informal_answer_parsed'] = answer_parsed[-1]
                    answer_succeeded_all.append((d, informal_answer, answer_parsed))
                else:
                    d['informalization']['informal_answer_parsed'] = None
                    answer_failed_all.append((d, informal_answer, answer_parsed))
        
        print(C.Counter('Unparsable Problem-Solving Question' if (d['informalization']['problem_type'] == 'Problem-Solving Question' and d['informalization']['informal_answer_parsed'] is None) else d['informalization']['problem_type'] for d in data))
        with open(osp.join(data_root, 'fixed', p), 'w') as f:
            for d in data:
                f.write(json.dumps(d, ensure_ascii=False) + '\n')

sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913-valid_samples.jsonl
len(potentially_wrong_list)=0
Counter({'Problem-Solving Question': 1345, 'Proof Question': 1258, 'Unparsable Problem-Solving Question': 131})
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913.staged-valid_samples.jsonl
len(potentially_wrong_list)=577
Counter({'Proof Question': 1125, 'Problem-Solving Question': 1093, 'Unparsable Problem-Solving Question': 122})
MUSTARDSauce_lean4_parsed-nocomment-valid_samples.jsonl
len(potentially_wrong_list)=88
Counter({'Problem-Solving Question': 2765, 'Unparsable Problem-Solving Question': 600, 'Proof Question': 426})
autoformalization_pg_kimina7b-PromptCoT-DS_kimina7b-valid_samples.jsonl
len(potentially_wrong_list)=26
Counter({'Proof Question': 230, 'Problem-Solving Question': 224, 'Unparsable Problem-Solving Question': 3})
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.prob

In [4]:
data_root = '/home/ma-user/workspace/formal_problem_generation/output_tmp/output/informalizations/processed'
for p in os.listdir(data_root):
    if osp.isfile(osp.join(data_root, 'fixed', p)):
        with open(osp.join(data_root, 'fixed', p), 'r') as f:
            data = [json.loads(l) for l in f.readlines()]

        potentially_wrong_list = [d for d in data if d['informalization']['problem_type'] == 'Problem-Solving Question' and  \
            any(t in d['informalization']['informal_problem'].lower() for t in ['prove', 'show'])]
        assert len(potentially_wrong_list) == 0
        
        data_informal_problem_solving = []

        for d in data:
            if d['informalization']['problem_type'] == 'Problem-Solving Question' and d['informalization']['informal_answer_parsed'] is not None:
                data_informal_problem_solving.append({
                    "conversation":[
                        {
                            # qwen25-math-cot
                            "system": r'Please reason step by step, and put your final answer within \boxed{}.',
                            "input": d['informalization']['informal_problem'],
                            "output": d['informalization']['informal_solution']
                        }
                    ]
                })
            else:
                data_informal_problem_solving.append({
                    "conversation":[
                        {
                            # qwen25-math-cot
                            "system": 'Please reason step by step.',
                            "input": d['informalization']['informal_problem'],
                            "output": d['informalization']['informal_solution']
                        }
                    ]
                })
        print(p, len(data_informal_problem_solving))
        with open(osp.join(data_root, 'data_informal_problem_solving', p), 'w') as f:
            for d in data_informal_problem_solving:
                f.write(json.dumps(d, ensure_ascii=False) + '\n')

sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913-valid_samples.jsonl 2734
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913.staged-valid_samples.jsonl 2340
MUSTARDSauce_lean4_parsed-nocomment-valid_samples.jsonl 3791
autoformalization_pg_kimina7b-PromptCoT-DS_kimina7b-valid_samples.jsonl 457
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch-valid_samples.jsonl 2726
autoformalization_pg_kimina7b-ScaleQuest-Math_kimina7b-valid_samples.jsonl 2035
autoformalization_pg_kimina7b-PromptCoT-QwQ_kimina7b-valid_samples.jsonl 1024
MUSTARDSauce_lean4_parsed-valid_samples.jsonl 3791
sft_wg_starified-Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack-valid_samples.jsonl 1164


In [5]:
print_xtuner_sample(random.choice(data_informal_problem_solving))

<SYSTEM>
Please reason step by step, and put your final answer within \boxed{}.
</SYSTEM>
<INPUT>
Let \( f \) be a function from real numbers to real numbers such that for all real numbers \( x \) and \( y \),
\[
f(x \cdot y + 1) = f(x) \cdot f(y) - f(y) - x + 2.
\]
Find the number of ordered pairs \( (a, b) \) of positive integers such that \( 2a + b = 100 \).
</INPUT>
<OUTPUT>
**Step 1: Understand the Problem**

We are to count the number of pairs \( (a, b) \) of positive integers (i.e., \( a > 0 \), \( b > 0 \)) satisfying the equation:
\[
2a + b = 100.
\]

**Step 2: Reformulate the Set**

The set of all such pairs is:
\[
S = \{(a, b) \in \mathbb{N} \times \mathbb{N} \mid a > 0, b > 0, 2a + b = 100\}.
\]

**Step 3: Determine the Range of \( a \) and \( b \)**

From \( 2a + b = 100 \) and \( b > 0 \), we have:
\[
2a < 100 \implies a < 50.
\]
Since \( a > 0 \), \( a \) ranges from 1 to 49.

Also, \( b = 100 - 2a \). Since \( b > 0 \), \( 100 - 2a > 0 \implies a < 50 \), which is alrea

In [6]:
data_root = '/home/ma-user/workspace/formal_problem_generation/output_tmp/output/informalizations/processed'
for p in os.listdir(data_root):
    if osp.isfile(osp.join(data_root, 'fixed', p)):
        with open(osp.join(data_root, 'fixed', p), 'r') as f:
            data = [json.loads(l) for l in f.readlines()]

        potentially_wrong_list = [d for d in data if d['informalization']['problem_type'] == 'Problem-Solving Question' and  \
            any(t in d['informalization']['informal_problem'].lower() for t in ['prove', 'show'])]
        assert len(potentially_wrong_list) == 0
        
        data_informal_problem_solving_onlypsq = []

        for d in data:
            if d['informalization']['problem_type'] == 'Problem-Solving Question' and d['informalization']['informal_answer_parsed'] is not None:
                data_informal_problem_solving_onlypsq.append({
                    "conversation":[
                        {
                            # qwen25-math-cot
                            "system": r'Please reason step by step, and put your final answer within \boxed{}.',
                            "input": d['informalization']['informal_problem'],
                            "output": d['informalization']['informal_solution']
                        }
                    ]
                })
        print(p, len(data_informal_problem_solving_onlypsq))
        with open(osp.join(data_root, 'data_informal_problem_solving_onlypsq', p), 'w') as f:
            for d in data_informal_problem_solving_onlypsq:
                f.write(json.dumps(d, ensure_ascii=False) + '\n')

sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913-valid_samples.jsonl 1345
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913.staged-valid_samples.jsonl 1093
MUSTARDSauce_lean4_parsed-nocomment-valid_samples.jsonl 2765
autoformalization_pg_kimina7b-PromptCoT-DS_kimina7b-valid_samples.jsonl 224
sft_ar_v3-Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch-valid_samples.jsonl 1326
autoformalization_pg_kimina7b-ScaleQuest-Math_kimina7b-valid_samples.jsonl 1886
autoformalization_pg_kimina7b-PromptCoT-QwQ_kimina7b-valid_samples.jsonl 683
MUSTARDSauce_lean4_parsed-valid_samples.jsonl 2744
sft_wg_starified-Goedel-Prover-V2-8B.Numina-Lean.whole_statement_generatior.nopack-valid_samples.jsonl 504


In [ ]:
# answer_failed_all = []
# answer_succeeded_all = []

# data_root = '/home/ma-user/workspace/formal_problem_generation/output_tmp/output/informalizations/processed'
# for p in tqdm(os.listdir(data_root)):
#     if osp.isfile(osp.join(data_root, 'fixed', p)):
#         with open(osp.join(data_root, 'fixed', p), 'r') as f:
#             data = [json.loads(l) for l in f.readlines()]

#         potentially_wrong_list = [d for d in data if d['informalization']['problem_type'] == 'Problem-Solving Question' and  \
#             any(t in d['informalization']['informal_problem'].lower() for t in ['prove', 'show'])]
#         assert len(potentially_wrong_list) == 0
#         for d in data:
#             if d['informalization']['problem_type'] == 'Problem-Solving Question':
#                 informal_answer = d['informalization']['informal_answer'].strip()
#                 if informal_answer.startswith(r'\boxed{'):
#                     informal_answer = informal_answer[len(r'\boxed{'):-1]
#                 answer_parsed = parse(informal_answer)
                
#                 if not (len(answer_parsed) == 2 and any(t in str(type(answer_parsed[0]).__module__) for t in ['sympy.core.numbers', 'latex2sympy2_extended.sets', 'sympy.sets', 'sympy.core.mul', 'sympy.core.add', 'sympy.core.power', 'sympy.matrices', 'sympy.functions'])):
#                     answer_failed_all.append((d, informal_answer, answer_parsed))
#                     answer_parsed = parse(d['informalization']['informal_solution'])
                
#                 if len(answer_parsed) == 2 and any(t in str(type(answer_parsed[0]).__module__) for t in ['sympy.core.numbers', 'latex2sympy2_extended.sets', 'sympy.sets', 'sympy.core.mul', 'sympy.core.add', 'sympy.core.power', 'sympy.matrices', 'sympy.functions']):
#                     answer_succeeded_all.append((d, informal_answer, answer_parsed))
#                 else:
#                     answer_failed_all.append((d, informal_answer, answer_parsed))

#         with open(osp.join(data_root, 'fixed', p), 'w') as f:
#             for d in data:
#                 f.write(json.dumps(d, ensure_ascii=False) + '\n')

100%|██████████| 13/13 [00:45<00:00,  3.49s/it]


In [118]:
C.Counter(None if len(a[-1]) < 2 else str(type(a[-1][0]).__module__) for a in answer_failed_all)

Counter({None: 1313,
         'sympy.core.symbol': 558,
         'sympy.core.relational': 322,
         'sympy.core.containers': 91,
         'latex2sympy2_extended.logic': 69,
         'None': 9})

In [119]:
C.Counter(None if len(a[-1]) < 2 else str(type(a[-1][0])) for a in answer_failed_all)

Counter({None: 1313,
         "<class 'sympy.core.symbol.Symbol'>": 558,
         "<class 'sympy.core.relational.Equality'>": 229,
         "<class 'sympy.core.containers.Tuple'>": 91,
         'And': 69,
         "<class 'sympy.core.relational.StrictLessThan'>": 45,
         "<class 'sympy.core.relational.StrictGreaterThan'>": 31,
         "<class 'sympy.core.relational.LessThan'>": 7,
         "<class 'sympy.core.relational.GreaterThan'>": 6,
         "<class 'sympy.core.relational.Unequality'>": 4,
         'f': 4,
         'C': 2,
         '\\text{dist}': 2,
         'p': 1})

In [120]:
C.Counter(None if len(a[-1]) < 2 else str(type(a[-1][0]).__module__) for a in answer_succeeded_all)

Counter({'sympy.core.numbers': 12037,
         'latex2sympy2_extended.sets': 565,
         'sympy.core.mul': 471,
         'sympy.core.add': 309,
         'sympy.core.power': 103,
         'sympy.sets.sets': 46,
         'sympy.functions.elementary.exponential': 5,
         'sympy.functions.elementary.trigonometric': 4,
         'sympy.matrices.dense': 4,
         'sympy.functions.elementary.complexes': 1,
         'sympy.functions.combinatorial.factorials': 1})

In [121]:
C.Counter(None if len(a[-1]) < 2 else str(type(a[-1][0])) for a in answer_succeeded_all)

Counter({"<class 'sympy.core.numbers.Integer'>": 8922,
         "<class 'sympy.core.numbers.Rational'>": 1210,
         "<class 'sympy.core.numbers.Float'>": 755,
         "<class 'latex2sympy2_extended.sets.FiniteSet'>": 565,
         "<class 'sympy.core.numbers.One'>": 508,
         "<class 'sympy.core.mul.Mul'>": 471,
         "<class 'sympy.core.numbers.Zero'>": 455,
         "<class 'sympy.core.add.Add'>": 309,
         "<class 'sympy.core.power.Pow'>": 103,
         "<class 'sympy.core.numbers.Half'>": 101,
         "<class 'sympy.core.numbers.NegativeOne'>": 85,
         "<class 'sympy.sets.sets.Interval'>": 46,
         'log': 4,
         "<class 'sympy.matrices.dense.MutableDenseMatrix'>": 4,
         'sin': 1,
         'atan': 1,
         'cos': 1,
         "<class 'sympy.core.numbers.ImaginaryUnit'>": 1,
         'exp': 1,
         'Abs': 1,
         'asin': 1,
         'binomial': 1})

In [106]:
C.Counter(None if len(a[-1]) < 2 else str(type(a[-1][1])) for a in answer_succeeded_all)

Counter({"<class 'str'>": 12570})

In [95]:
for a in answer_failed_all:
    if len(a[-1]) == 2 and 'log' in str(type(a[-1][0])):
        
        print(a[-1][0], a[-1][1])
        print(a[1].strip())
        # print(a[0]['informalization']['informal_problem'])
        # print(a[0]['formal_statement'])
        # print(a[0]['informalization']['informal_problem'])
        print()
        print()
        break

log(1/x, 2) \log_2 ( \frac{1}{x} )
\[
\log_2 \left( \frac{1}{x} \right)
\]




In [97]:
type(a[-1][0]).__module__

'sympy.functions.elementary.exponential'

{'header': '',
 'formal_statement': 'example\n(d1 : ℝ)\n(h₀ : d1 = 3)\n(d2 : ℝ)\n(h₁ : d2 = 4)\n(v1 : ℝ)\n(h₂ : v1 = 4)\n(v2 : ℝ)\n(h₃ : v2 = 3)\n: (d1 + d2) / (d1 / v1 + d2 / v2) = 3.36\n:= sorry',
 'formal_proof': 'have h₄: \n((d1 + d2) / (d1 / v1 + d2 / v2)) = 3.36  := by {\n  norm_num [h₀, h₁, h₂, h₃]\n}\n\nexact h₄',
 'condition': [['problem_type', 'Calculus'], ['source', 'math_train']],
 'index': 4999,
 'src': 'sft_ar_v3/Goedel-Prover-V2-8B.Numina-Lean-reasseblmed.39509.problem_generator.nopack.3epoch.0913',
 'responses': ['## Problem-Solving Question\nLet \\( d_1 = 3 \\), \\( d_2 = 4 \\), \\( v_1 = 4 \\), and \\( v_2 = 3 \\) be real numbers. Compute the value of the expression:\n\\[\n\\frac{d_1 + d_2}{\\frac{d_1}{v_1} + \\frac{d_2}{v_2}}.\n\\]\n\n## Answer\n3.36\n\n## Solution\n**Step 1: Substitute the given values into the expression.**  \nGiven:\n\\[\nd_1 = 3, \\quad d_2 = 4, \\quad v_1 = 4, \\quad v_2 = 3.\n\\]\nSubstitute:\n\\[\n\\frac{3 + 4}{\\frac{3}{4} + \\frac{4}{3}} = \

In [68]:
parse(r'\( (3.5, 2) \)')

[(3.5, 2), '(3.5, 2)']

In [55]:
a[-1]

[3.36000000000000, '3.36']

In [44]:
d, informal_answer, answer_parsed = random.choice(failed_list)
print(informal_answer, answer_parsed)

Yes []


In [48]:
print(d['formal_statement'])
print(d['informalization']['informal_solution'])

example
(total_distance : ℝ)
(h : total_distance = 240)
(first_hour : ℝ)
(hf : first_hour = 24 / 60)
(second_hour : ℝ)
(hs : second_hour = 20 / 60)
(third_hour : ℝ)
(ht : third_hour = (total_distance - 60) / 100)
: first_hour + second_hour < third_hour
:= sorry
**Step 1: Write down the given values.**  
Total distance: \( 240 \) km.  
First hour distance: \( 24 \) km.  
Second hour distance: \( 20 \) km.  
Third hour time: \( \frac{240 - 60}{100} \) hours.

**Step 2: Compute the times for the first two hours.**  
Since speed is constant per hour (implied by "hour" distances), the time for the first hour is \( \frac{24}{60} = 0.4 \) hours.  
The time for the second hour is \( \frac{20}{60} = \frac{1}{3} \approx 0.333 \) hours.  
Sum: \( 0.4 + \frac{1}{3} = \frac{2}{5} + \frac{1}{3} = \frac{6}{15} + \frac{5}{15} = \frac{11}{15} \approx 0.733 \) hours.

**Step 3: Compute the time for the third hour.**  
\( \frac{240 - 60}{100} = \frac{180}{100} = 1.8 \) hours.

**Step 4: Compare the two t

In [29]:
parse('Yes, the answer is correct, \\boxed{Correct}')

[E*c*c*o*r*r*t, 'Correct']

In [30]:
type(parse('Yes, the answer is correct, \\boxed{Correct}')[0])

sympy.core.mul.Mul